## Workflow Schedules
#### DynamoDB Table - wfm-&lt;teamName&gt;-AMCSchedules-&lt;env&gt;


In [ ]:
import boto3
import json
import pandas as pd
from atsclientslibraries.workflow_schedules import workflowSchedules

## Global Configs
#### Edit the following configuration for your particular AMC Instance.

In [ ]:
## The Team Name configured in the Data Lake platform
# This is the same value passed in the 'data_lake_parameters' team of ddk.json file
TEAM_NAME = "demoteam"

aws_region = 'us-east-1'
ENV="dev"
customerId = 'democustomer'
if aws_region == '':
    aws_region = str(boto3.Session().region_name)
print("Region : " + aws_region)

## Structure of AMC Workflow Schedule and it's description. They are stored in AMCSchedules DynamoDB Table.

```
{
  "customerID":<ID of Customer. This has to be unique for each customer>,
  ""Name":<Name of the Workflow>"
  "Description":<Description of the Workflow>,
  "automaticDeploySchedule":<Set to True if the workflow needs to be automatically deployed from the Library to the Schedule Table, or set to False if not>,
  "ScheduleExpression":<Schedule expression of the Workflow. See below for more details.>,
  "State":<ENABLED or DISABLED; If set to DISABLED, the schedule will not be executed>,
  "Input": {
      "payload": {
          "timeWindowEnd":<End Date of the report for e.g today(-1) will set the end time to be 1 day back form current time>,
          "timeWindowStart":Start date of the report for e.g today(-15) will set the start time to be 15 days back from current time>,
          "timeWindowType":<EXPLICIT>,
          "workflow_executed_date":<e.g now()>,
          "workflowId":<ID of the workflow. This has to be unique for each workflow>
      }
  }   
}
```

## Schedule Expressions

```
Two Types of Schedule Expressions:
1) Cron

Cron Expression Format: cron(Minutes Hours Day-of-month Month Day-of-week Year)
Use Cron Expression if a workflow needs to be scheduled at a specific day and time for e.g cron(15 10 * * ? *) means the workflow will be scheduled for 10:15(UTC) Everyday or cron(0 9 ? * 2#1 *) means the workflow will be scheduled for 9:00 AM on the first Monday of each month. For details refer https://docs.aws.amazon.com/AmazonCloudWatch/latest/events/ScheduledEvents.html

2) Custom (Recommended)
Custom Expressin Format: custom({H/D/W/M} {Day-of-the-week/month} {Hour-of-the-Day}), where H=Hourly, D=Daily, W=Weekly and M=Monthly.
Use custom Expression if there is no hard requirement for a workflow to run at specific time. For e.g, custom(D * 14) means workflow will be scheduled to run Daily at sometime between 14:00 and 14:59 UTC, custom(W 2 8) means it will be scheduled to run Weekly on every 2nd day of the week at sometime between 08:00 and 08:59 UTC, custom(H * *) will schedule it to run every hour.
This is the recommended approach as it utilizes less resources.
```

### Workflow Schedules are automatically loaded from the workflow library if the automaticDeploySchedule is set True. This Notebook can help add new custom workflow schedules or update any existing ones such as change the schedule time. Below is a sample Workflow Schedule record. Use this sample to create or update Workflows Schedules.

In [ ]:
# Workflow record
workflow =  {
  "automaticDeploySchedule": False,
  "customerId": customerId,
  "Description": "Runs the exposure_group_analysisn workflow at 11:00 GMT / 9:00am EST daily. ",
  "Input": {
    "payload": {
      "timeWindowEnd": "today(-2)",
      "timeWindowStart": "today(-3)",
      "timeWindowType": "EXPLICIT",
      "workflow_executed_date": "now()",
      "workflowId": "exposure_group_analysis_v1"
    }
  },
  "Name": f'wfm-{customerId}-exposure_group_analysis_v1',
  "ScheduleExpression": "custom(D * 11)",
  "State": "ENABLED"
}

print(json.dumps(workflow, indent=4))

### Create/Update an AMC Workflow Schedule:

In [ ]:
dynamodb_resp_wr = workflowSchedules.set_workflow_schedule(workflow_details=workflow, TEAM_NAME=TEAM_NAME, ENV=ENV)
dynamodb_resp_wr

### Retrieve AMC Workflow Schedules:

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)

df = workflowSchedules.get_workflow_schedule(TEAM_NAME=TEAM_NAME, ENV=ENV)
df.head()

## Delete a Workflow Schedule:

In [ ]:
wf_name = f'wfm-{customerId}-exposure_group_analysis_v1'
dynamodb_resp_dl = workflowSchedules.delete_workflow_schedule(customerId=customerId, workflowName=wf_name, TEAM_NAME=TEAM_NAME, ENV=ENV)
dynamodb_resp_dl